In [6]:
import numpy as np
from quasinet.qnet import load_qnet
from quasinet.qnet import save_qnet
from quasinet.qsampling import qsample
from quasinet.qnet import qdistance, qdistance_matrix
from quasinet.qnet import Qnet


## Create, load and save quasinet model

In [3]:
#MODELNAME='global2.joblib.gz'
#model=load_qnet(MODELNAME,gz=True)

#MODELNAME='catptsd.joblib'
#model=load_qnet(MODELNAME,gz=False)

model=Qnet(feature_names=['X'+str(x) for x in range(15)],alpha=.1)
X=np.random.randint(10, size=(100, 15)).astype('str')
model.fit(X)

save_qnet(model,'model.joblib.gz',gz=True)

## Compute qdistance between samples

In [4]:
qdistance(X[0],X[1],model,model)

0.03303279692657593

In [7]:
qdistance_matrix(X[:10],X[:5],model,model)

array([[0.        , 0.0330328 , 0.03995392, 0.05202185, 0.03072514],
       [0.0330328 , 0.        , 0.02860261, 0.03291197, 0.03568944],
       [0.03995392, 0.02860261, 0.        , 0.03096436, 0.03521861],
       [0.05202185, 0.03291197, 0.03096436, 0.        , 0.02522709],
       [0.03072514, 0.03568944, 0.03521861, 0.02522709, 0.        ],
       [0.05800043, 0.07859987, 0.06978143, 0.07361493, 0.05571045],
       [0.04710786, 0.02475045, 0.0286777 , 0.03515892, 0.04094038],
       [0.03588783, 0.04075767, 0.03343664, 0.03288908, 0.03864915],
       [0.04187565, 0.04685019, 0.03823567, 0.05121765, 0.03591758],
       [0.04835767, 0.02712291, 0.02779251, 0.03354772, 0.03868481]])

## Draw qsample to generate samples which follow the inferred probabilistic constraints

We call these the "core samples" if `steps` is large

In [8]:
qsample(X[0],model,steps=10)

array(['3', '0', '7', '4', '5', '6', '3', '5', '5', '9', '0', '5', '1',
       '9', '6'], dtype='<U21')

## Evaluate fetature importances using Shap anlaysis

Using ,fast_estimate=False using kernel explainer, otherwise 
a catboost based tree explainer, which is significantluy faster,
but probably less accurate

In [9]:
from quasinet.feature_importance import getShap

In [10]:
sf,index_=getShap(model,fast_estimate=False)

  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
sf

,shapval
X14,-0.022672
X13,0.005082
X1,-0.004271
X2,0.003941
X10,0.003169
X3,0.002976
X9,-0.002956
X7,0.002587
X4,0.001870
X5,-0.000973


## Compute metric tensor and curvature

In [15]:
from quasinet.curvature import compute_metric_tensor, compute_curvature
import numpy as np
from quasinet.utils import getNull

In [16]:
p=np.array(model.predict_distributions(getNull(model,strtype='U5')))[index_[:10]]

In [17]:
compute_metric_tensor(p,delta=0.001)

array([[0.00000000e+00, 1.54359397e-04, 1.56401307e-04, 1.73931175e-04,
        1.53612026e-04, 1.48305277e-04, 2.15271750e-04, 1.28228321e-04,
        1.85878971e-04, 1.23501624e-04],
       [1.54359397e-04, 0.00000000e+00, 1.34462539e-04, 1.49533452e-04,
        1.32064517e-04, 1.27502157e-04, 1.85075090e-04, 1.10241441e-04,
        1.59805304e-04, 1.06177769e-04],
       [1.56401307e-04, 1.34462539e-04, 0.00000000e+00, 1.51511523e-04,
        1.33811504e-04, 1.29188792e-04, 1.87523316e-04, 1.11699747e-04,
        1.61919254e-04, 1.07582319e-04],
       [1.73931175e-04, 1.49533452e-04, 1.51511523e-04, 0.00000000e+00,
        1.48809447e-04, 1.43668610e-04, 2.08541420e-04, 1.24219347e-04,
        1.80067587e-04, 1.19640427e-04],
       [1.53612026e-04, 1.32064517e-04, 1.33811504e-04, 1.48809447e-04,
        0.00000000e+00, 1.26884823e-04, 1.84179002e-04, 1.09707679e-04,
        1.59031565e-04, 1.05663682e-04],
       [1.48305277e-04, 1.27502157e-04, 1.29188792e-04, 1.43668610e-04,
   

In [18]:
compute_curvature(p,delta=0.001)

100%|██████████| 10/10 [00:00<00:00, 49113.63it/s]


412538.3136198949